In [ ]:
import numpy as np

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping , ReduceLROnPlateau
from nltk.stem import PorterStemmer , WordNetLemmatizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report
from tensorflow.keras.layers import Conv1D , Flatten , Dense , Embedding , Dropout , MaxPooling1D
from tensorflow.keras import Sequential

In [ ]:
nltk.download("stopwords")
nltk.download("words")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!kaggle datasets download -d pashupatigupta/emotion-detection-from-text

Dataset URL: https://www.kaggle.com/datasets/pashupatigupta/emotion-detection-from-text
License(s): CC0-1.0
  0% 0.00/1.56M [00:00<?, ?B/s]
100% 1.56M/1.56M [00:00<00:00, 162MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/emotion-detection-from-text.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
df=pd.read_csv('/content/tweet_emotions.csv')

In [ ]:
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [ ]:
df.shape

(40000, 3)

In [ ]:
df.isna().sum()

,0
tweet_id,0
sentiment,0
content,0


In [ ]:
df.duplicated().sum()

0

In [ ]:
df['sentiment'].value_counts(ascending = True)

,count
sentiment,
anger,110
boredom,179
enthusiasm,759
empty,827
hate,1323
relief,1526
fun,1776
surprise,2187
love,3842


In [ ]:
label_to_remove=['anger','boredom','enthusiasm','empty','relief']

In [ ]:
df = df[~df['sentiment'].isin(label_to_remove)]

In [ ]:
df['sentiment'].value_counts(ascending = True)

,count
sentiment,
hate,1323
fun,1776
surprise,2187
love,3842
sadness,5165
happiness,5209
worry,8459
neutral,8638


In [ ]:
df.shape

(36599, 3)

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Remove mentions (usernames starting with @)
    text = re.sub(r'@\w+', '', text)

    # Lowercase the text
    text = text.lower()

    # Remove punctuation (but keep sentiment-related punctuation like "!" and "?")
    text = re.sub(r"[^\w\s!?]", "", text)

    # Tokenize the text (optional stopword removal)
    tokens = word_tokenize(text)

    # Optionally remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Rejoin tokens
    return " ".join(tokens)

In [ ]:
# Apply preprocessing to each row in the 'content' column
df['cleaned_content'] = df['content'].apply(preprocess_text)

<ipython-input-27-5b5ef62c5c44>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_content'] = df['content'].apply(preprocess_text)


In [ ]:
from transformers import BertTokenizer

In [ ]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Example: Tokenize using BERT
df['bert_tokens'] = df['cleaned_content'].apply(lambda text: tokenizer(text, padding='max_length', truncation=True, max_length=128))


In [ ]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label encoding for sentiments
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentiment'])


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['bert_tokens'].tolist(),
      y,
    test_size=0.2,
)

In [ ]:
train_dataset = Dataset.from_dict({
    'input_ids': [x['input_ids'] for x in train_texts],
    'attention_mask': [x['attention_mask'] for x in train_texts],
    'labels': list(train_labels)
})

test_dataset = Dataset.from_dict({
    'input_ids': [x['input_ids'] for x in test_texts],
    'attention_mask': [x['attention_mask'] for x in test_texts],
    'labels': list(test_labels)
})

In [ ]:
# Load BERT for sequence classification with the correct number of output classes
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=16,   # batch size for training
    per_device_eval_batch_size=64,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"       # evaluate after each epoch
)

# Define Trainer
trainer = Trainer(
    model=model,                         # the BERT model
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.634700,1.603521
2,1.419800,1.606497
3,1.314400,1.765702


TrainOutput(global_step=5490, training_loss=1.4271193627668426, metrics={'train_runtime': 2128.8829, 'train_samples_per_second': 41.26, 'train_steps_per_second': 2.579, 'total_flos': 5778032697391104.0, 'train_loss': 1.4271193627668426, 'epoch': 3.0})

In [ ]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 1.7657018899917603,
 'eval_runtime': 46.8954,
 'eval_samples_per_second': 156.092,
 'eval_steps_per_second': 2.452,
 'epoch': 3.0}